## Reserach

In [7]:
from pathlib import Path
from data_model import Network, AlgorithmParameters
from parsing import NetworkParser
from pathlib import Path
from evolution import Evolution
from typing import List, Tuple
from util import myThread
import math

In [8]:
populations = [10, 25, 50, 100]
crossover_probs = [round(i * 0.1, 2) for i in range(1, 2)]
mutation_probs = [round(i * 0.1, 2) for i in range(0, 11)]
mutation_power = [round(i * 0.1, 2) for i in range(0, 11)]


param_space : List[Tuple[int, float, float, float]] = []

for population in populations:
    for cross_prob in crossover_probs:
        for mut_prob in mutation_probs:
            for mut_pwr in mutation_power:
                param_space.append((population, cross_prob, mut_prob, mut_pwr))


In [19]:
parser = NetworkParser(Path("polska/polska.xml"))

net = Network(
    demands=parser.demands(),
    links=parser.links(),
    modularity=10,
    aggregation=False
)

results: List[Tuple[float, Tuple[int, float, float, float]]] = []
counter = 1

param_space = param_space[0:10]
best = (math.inf, None)
for param_combination in param_space:
    params = AlgorithmParameters(
        population_size=param_combination[0],
        crossover_prob=param_combination[1],
        tournament_size=2,
        mutation_prob=param_combination[2],
        mutation_power=param_combination[3],
        mutation_range=1,
        target_fitness=0,
        max_epochs=5000,
        stale_epochs_limit=500,
    )

    evo = Evolution(
        network=net,
        parameters=params
    )

    evo.run()
    value = evo.get_result()['modules']
    
    if value < best[0]:
        best = (value, param_combination)

    print("Run #" + str(counter) + " best :" + str(best[0]) + " best params : " + str(best[1]) + " current params : " + str(param_combination))
    counter += 1


Run #1 best :4762 best params : (10, 0.0, 0.0, 0.0) current params : (10, 0.0, 0.0, 0.0)
Run #2 best :4762 best params : (10, 0.0, 0.0, 0.0) current params : (10, 0.0, 0.0, 0.1)
Run #3 best :4762 best params : (10, 0.0, 0.0, 0.0) current params : (10, 0.0, 0.0, 0.2)
Run #4 best :4762 best params : (10, 0.0, 0.0, 0.0) current params : (10, 0.0, 0.0, 0.3)
Run #5 best :4762 best params : (10, 0.0, 0.0, 0.0) current params : (10, 0.0, 0.0, 0.4)
Run #6 best :4659 best params : (10, 0.0, 0.0, 0.5) current params : (10, 0.0, 0.0, 0.5)
Run #7 best :4659 best params : (10, 0.0, 0.0, 0.5) current params : (10, 0.0, 0.0, 0.6)
Run #8 best :4659 best params : (10, 0.0, 0.0, 0.5) current params : (10, 0.0, 0.0, 0.7)
Run #9 best :4659 best params : (10, 0.0, 0.0, 0.5) current params : (10, 0.0, 0.0, 0.8)
Run #10 best :4659 best params : (10, 0.0, 0.0, 0.5) current params : (10, 0.0, 0.0, 0.9)


In [12]:
thread_number = 10

net = Network(
    demands=parser.demands(),
    links=parser.links(),
    modularity=10,
    aggregation=False
)

param_groups = len(param_space) // thread_number
threads = [myThread(param_space[param_groups * id : param_groups * (id + 1)], str(id)) for id in range(0, thread_number)]

for thread in threads:
    thread.start()

for thread in threads:
    thread.join()

best_fitness: float = math.inf
best_params: Tuple[int, float, float, float] = (0, 0.0, 0.0, 0.0)

for thread in threads:
    if best_fitness < thread.best_value:
        best_params = thread.best_params
        best_fitness = thread.best_value

print(best_params)
print(best_fitness)

Exception in thread Thread-60:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1009, in _bootstrap_inner
    self.run()
  File "/home/jzielins/sem7/POP/project/POP-TeleNet/util.py", line 32, in run
    with open('./results/thread' + self.id, 'w') as file:
FileNotFoundError: [Errno 2] No such file or directory: 'results/thread1'
Exception in thread Thread-61:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1009, in _bootstrap_inner
    self.run()
  File "/home/jzielins/sem7/POP/project/POP-TeleNet/util.py", line 32, in run
    with open('./results/thread' + self.id, 'w') as file:
FileNotFoundError: [Errno 2] No such file or directory: 'results/thread2'
Exception in thread Thread-62:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1009, in _bootstrap_inner
    self.run()
  File "/home/jzielins/sem7/POP/project/POP-TeleNet/util.py", line 32, in run
Exception in thread Thread-59:
Traceb

(0, 0.0, 0.0, 0.0)
inf
